In [1]:
import os

from Utils import FileUtils as filu
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 40)
pd.options.mode.chained_assignment = None
from bs4 import BeautifulSoup
import requests
import re
from requests_html import HTMLSession
import numpy as np

ModuleNotFoundError: No module named 'Utils'

In [ ]:
s = HTMLSession()

In [ ]:
def getdata(url):
    headers = {"User-Agent":
              "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
              "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"}
    r = s.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

In [ ]:
url = "https://whoicf2.whoi.edu/science/B/whalesounds/index.cfm"
base_url = "https://whoicf2.whoi.edu"

soup = getdata(url)

In [ ]:
options = soup.find("form", {"name": "jump1"}).find_all("option")

In [ ]:
pages = list({option.text.replace("\n", "").replace("\t", "")[1:]:
                  option["value"] for option in options}.items())[1:]

In [ ]:
df = pd.DataFrame(columns=["species", "filename"])
pages = list({option.text.replace("\n", "").replace("\t", "")[1:]:
                  option["value"] for option in options}.items())[1:]
first = True
meta_links = {}
for i, (whale_name, page_link) in enumerate(pages):
    print(whale_name)
    meta_links[whale_name] = []
    page = getdata(page_link)
    rec_num = 0
    for a in page.find("table").find_all("a", href=True):
        link = a["href"]
        if link.split(".")[-1] != "wav" and link.split(".")[-1] != "mp3":
            meta_link = base_url + "/science/B/whalesounds/" + link.split("'")[1]
            page = getdata(meta_link)
            table = page.find_all("table")[1]
            #columns = table.find("tr").text.split("\n")[:2]
            

            cols = []
            filename = whale_name + " " + str(rec_num) + ".wav"
            rows = [whale_name, filename]            
            
            for j, data in enumerate(table.find_all("td")):
                if j % 2 == 0: 
                    cols.append(data.text)
                else:
                    rows.append(data.text)            
            
            if first:
                for col in cols:
                    df[col] = np.nan
                first = False
            
            df.loc[len(df.index)] = rows
            rec_num = rec_num + 1
df.to_csv(path_or_buf = r"C:\datasets\whoi\meta.csv")